In [1]:
import tkinter as tk
from tkinter import messagebox
import math
import random
from functools import partial

HUMAN = "X"
AI = "O"


def evaluate_winner(board):
    lines = [
        (0, 1, 2), (3, 4, 5), (6, 7, 8),
        (0, 3, 6), (1, 4, 7), (2, 5, 8),
        (0, 4, 8), (2, 4, 6),
    ]
    for a, b, c in lines:
        if board[a] and board[a] == board[b] == board[c]:
            return board[a], (a, b, c)
    if all(board):
        return "DRAW", None
    return None, None

def minimax_alpha_beta(board, maximizing, alpha, beta, depth, max_depth):
    winner, _ = evaluate_winner(board)
    if winner == AI:
        return 10 - depth, None
    if winner == HUMAN:
        return -10 + depth, None
    if winner == "DRAW":
        return 0, None
    if max_depth is not None and depth >= max_depth:
        return 0, None  # depth-limited

    if maximizing:
        best_score = -math.inf
        best_moves = []
        for i in range(9):
            if not board[i]:
                board[i] = AI
                score, _ = minimax_alpha_beta(board, False, alpha, beta, depth + 1, max_depth)
                board[i] = None
                if score > best_score:
                    best_score = score
                    best_moves = [i]
                elif score == best_score:
                    best_moves.append(i)
                alpha = max(alpha, best_score)
                if beta <= alpha:
                    break
        return best_score, random.choice(best_moves) if best_moves else None
    else:
        best_score = math.inf
        best_moves = []
        for i in range(9):
            if not board[i]:
                board[i] = HUMAN
                score, _ = minimax_alpha_beta(board, True, alpha, beta, depth + 1, max_depth)
                board[i] = None
                if score < best_score:
                    best_score = score
                    best_moves = [i]
                elif score == best_score:
                    best_moves.append(i)
                beta = min(beta, best_score)
                if beta <= alpha:
                    break
        return best_score, random.choice(best_moves) if best_moves else None


class TicTacToeApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Tic-Tac-Toe (Alpha–Beta AI)")
        self.root.resizable(False, False)

        self.board = [None] * 9
        self.buttons = []
        self.move_history = []
        self.last_move_idx = None
        self.current_player = HUMAN
        self.game_over = False

        # Top bar
        self.top_frame = tk.Frame(root, padx=10, pady=8)
        self.top_frame.pack()

        self.status_var = tk.StringVar(value="Your turn (X)")
        tk.Label(self.top_frame, textvariable=self.status_var, font=("Segoe UI", 14)).grid(row=0, column=0, columnspan=3, sticky="w")

        tk.Button(self.top_frame, text="Reset", command=self.reset, width=8).grid(row=0, column=3, padx=(12, 0))
        tk.Button(self.top_frame, text="Undo", command=self.undo, width=8).grid(row=0, column=4, padx=(8, 0))

        self.ai_first_var = tk.IntVar(value=0)
        tk.Checkbutton(self.top_frame, text="AI vs AI", variable=self.ai_first_var, command=self.toggle_ai_vs_ai).grid(row=0, column=5, padx=(8, 0))

        # Difficulty
        tk.Label(self.top_frame, text="Difficulty:").grid(row=0, column=6, padx=(12, 0))
        self.difficulty_var = tk.StringVar(value="Hard")
        tk.OptionMenu(self.top_frame, self.difficulty_var, "Easy", "Medium", "Hard").grid(row=0, column=7)

        # Game grid
        self.grid_frame = tk.Frame(root, padx=10, pady=10, bg="#f5f5f5")
        self.grid_frame.pack()
        for r in range(3):
            for c in range(3):
                idx = 3 * r + c
                btn = tk.Button(
                    self.grid_frame,
                    text="",
                    font=("Segoe UI", 24, "bold"),
                    width=3,
                    height=1,
                    command=partial(self.on_click, idx)
                )
                btn.grid(row=r, column=c, padx=6, pady=6, ipadx=6, ipady=6)
                self.buttons.append(btn)

        self.footer = tk.Label(root, text="Alpha–Beta Pruning Tic-Tac-Toe", fg="#555")
        self.footer.pack(pady=(0, 8))

    def difficulty_depth(self):
        return {"Easy": 1, "Medium": 3, "Hard": None}[self.difficulty_var.get()]

    def toggle_ai_vs_ai(self):
        self.reset()
        if self.ai_first_var.get():
            self.current_player = HUMAN
            self.root.after(300, self.ai_vs_ai_step)

    def on_click(self, idx):
        if self.game_over or self.ai_first_var.get():
            return
        if self.board[idx] is not None:
            return
        self.place_mark(idx, HUMAN)
        self.after_move()

    def place_mark(self, idx, symbol):
        self.board[idx] = symbol
        self.move_history.append((idx, symbol))
        self.last_move_idx = idx
        self.update_buttons()

    def update_buttons(self):
        for i, btn in enumerate(self.buttons):
            btn.config(text=self.board[i] if self.board[i] else "", state="normal", bg="SystemButtonFace")
        if self.last_move_idx is not None:
            self.buttons[self.last_move_idx].config(bg="#fffa90")  # highlight last move
        for i, b in enumerate(self.buttons):
            if self.board[i] is not None:
                b.config(state="disabled")

    def after_move(self):
        winner, line = evaluate_winner(self.board)
        if winner:
            self.end_game(winner, line)
            return
        if not self.ai_first_var.get():
            self.status_var.set("AI thinking…")
            self.root.update_idletasks()
            self.root.after(300, self.ai_move)

    def ai_move(self):
        if self.game_over:
            return
        maximizing = (self.current_player == AI)
        _, move = minimax_alpha_beta(self.board, maximizing, -math.inf, math.inf, 0, self.difficulty_depth())
        if move is not None:
            self.place_mark(move, self.current_player)
        winner, line = evaluate_winner(self.board)
        if winner:
            self.end_game(winner, line)
            return
        self.current_player = HUMAN if self.current_player == AI else AI
        if self.ai_first_var.get():
            self.root.after(300, self.ai_vs_ai_step)
        else:
            self.status_var.set("Your turn (X)")

    def ai_vs_ai_step(self):
        if self.game_over:
            return
        maximizing = (self.current_player == AI)
        _, move = minimax_alpha_beta(self.board, maximizing, -math.inf, math.inf, 0, self.difficulty_depth())
        if move is not None:
            self.place_mark(move, self.current_player)
        winner, line = evaluate_winner(self.board)
        if winner:
            self.end_game(winner, line)
            return
        self.current_player = HUMAN if self.current_player == AI else AI
        self.root.after(300, self.ai_vs_ai_step)

    def end_game(self, winner, line):
        self.game_over = True
        if line:
            for i in line:
                self.buttons[i].config(bg="#b2ffb2" if winner != "DRAW" else "#ddd")
        if winner == "DRAW":
            self.status_var.set("It's a draw!")
        else:
            self.status_var.set(f"{winner} wins!")

    def reset(self):
        self.board = [None] * 9
        self.move_history = []
        self.last_move_idx = None
        self.current_player = HUMAN
        self.game_over = False
        self.update_buttons()
        self.status_var.set("Your turn (X)" if not self.ai_first_var.get() else "AI vs AI running...")

    def undo(self):
        if not self.move_history or self.game_over:
            return
        idx, _ = self.move_history.pop()
        self.board[idx] = None
        if self.move_history:
            self.last_move_idx = self.move_history[-1][0]
        else:
            self.last_move_idx = None
        self.current_player = HUMAN
        self.update_buttons()
        self.status_var.set("Move undone.")


if __name__ == "__main__":
    root = tk.Tk()
    app = TicTacToeApp(root)
    root.mainloop()


In [2]:
import tkinter as tk
from tkinter import messagebox
import math
import random
from functools import partial

HUMAN = "X"
AI = "O"


def evaluate_winner(board):
    lines = [
        (0, 1, 2), (3, 4, 5), (6, 7, 8),
        (0, 3, 6), (1, 4, 7), (2, 5, 8),
        (0, 4, 8), (2, 4, 6),
    ]
    for a, b, c in lines:
        if board[a] and board[a] == board[b] == board[c]:
            return board[a], (a, b, c)
    if all(board):
        return "DRAW", None
    return None, None


def minimax_alpha_beta(board, maximizing, alpha, beta, depth, max_depth):
    winner, _ = evaluate_winner(board)
    if winner == AI:
        return 10 - depth, None
    if winner == HUMAN:
        return -10 + depth, None
    if winner == "DRAW":
        return 0, None
    if max_depth is not None and depth >= max_depth:
        return 0, None  # depth-limited

    if maximizing:
        best_score = -math.inf
        best_moves = []
        for i in range(9):
            if not board[i]:
                board[i] = AI
                score, _ = minimax_alpha_beta(board, False, alpha, beta, depth + 1, max_depth)
                board[i] = None
                if score > best_score:
                    best_score = score
                    best_moves = [i]
                elif score == best_score:
                    best_moves.append(i)
                alpha = max(alpha, best_score)
                if beta <= alpha:
                    break
        return best_score, random.choice(best_moves) if best_moves else None
    else:
        best_score = math.inf
        best_moves = []
        for i in range(9):
            if not board[i]:
                board[i] = HUMAN
                score, _ = minimax_alpha_beta(board, True, alpha, beta, depth + 1, max_depth)
                board[i] = None
                if score < best_score:
                    best_score = score
                    best_moves = [i]
                elif score == best_score:
                    best_moves.append(i)
                beta = min(beta, best_score)
                if beta <= alpha:
                    break
        return best_score, random.choice(best_moves) if best_moves else None


class TicTacToeApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Tic-Tac-Toe (Alpha–Beta AI)")
        self.root.resizable(False, False)

        self.board = [None] * 9
        self.buttons = []
        self.move_history = []
        self.last_move_idx = None
        self.current_player = HUMAN
        self.game_over = False

        # Top bar
        self.top_frame = tk.Frame(root, padx=10, pady=8)
        self.top_frame.pack()

        self.status_var = tk.StringVar(value="Your turn (X)")
        tk.Label(self.top_frame, textvariable=self.status_var, font=("Segoe UI", 14)).grid(row=0, column=0, columnspan=3, sticky="w")

        tk.Button(self.top_frame, text="Reset", command=self.reset, width=8).grid(row=0, column=3, padx=(12, 0))
        tk.Button(self.top_frame, text="Undo", command=self.undo, width=8).grid(row=0, column=4, padx=(8, 0))

        self.ai_first_var = tk.IntVar(value=0)
        tk.Checkbutton(self.top_frame, text="AI vs AI", variable=self.ai_first_var, command=self.toggle_ai_vs_ai).grid(row=0, column=5, padx=(8, 0))

        # Difficulty
        tk.Label(self.top_frame, text="Difficulty:").grid(row=0, column=6, padx=(12, 0))
        self.difficulty_var = tk.StringVar(value="Hard")
        tk.OptionMenu(self.top_frame, self.difficulty_var, "Easy", "Medium", "Hard").grid(row=0, column=7)

        # Game grid
        self.grid_frame = tk.Frame(root, padx=10, pady=10, bg="#f5f5f5")
        self.grid_frame.pack()
        for r in range(3):
            for c in range(3):
                idx = 3 * r + c
                btn = tk.Button(
                    self.grid_frame,
                    text="",
                    font=("Segoe UI", 24, "bold"),
                    width=3,
                    height=1,
                    command=partial(self.on_click, idx)
                )
                btn.grid(row=r, column=c, padx=6, pady=6, ipadx=6, ipady=6)
                self.buttons.append(btn)

        self.footer = tk.Label(root, text="Alpha–Beta Pruning Tic-Tac-Toe", fg="#555")
        self.footer.pack(pady=(0, 8))

    def difficulty_depth(self):
        return {"Easy": 1, "Medium": 3, "Hard": None}[self.difficulty_var.get()]

    def toggle_ai_vs_ai(self):
        self.reset()
        if self.ai_first_var.get():
            self.current_player = HUMAN
            self.root.after(300, self.ai_vs_ai_step)

    def on_click(self, idx):
        if self.game_over or self.ai_first_var.get():
            return
        if self.board[idx] is not None:
            return
        self.place_mark(idx, HUMAN)
        self.after_move()

    def place_mark(self, idx, symbol):
        self.board[idx] = symbol
        self.move_history.append((idx, symbol))
        self.last_move_idx = idx
        self.update_buttons()

    def update_buttons(self):
        for i, btn in enumerate(self.buttons):
            btn.config(text=self.board[i] if self.board[i] else "", state="normal", bg="SystemButtonFace")
        if self.last_move_idx is not None:
            self.buttons[self.last_move_idx].config(bg="#fffa90")  # highlight last move
        for i, b in enumerate(self.buttons):
            if self.board[i] is not None:
                b.config(state="disabled")

    def after_move(self):
        winner, line = evaluate_winner(self.board)
        if winner:
            self.end_game(winner, line)
            return
        if not self.ai_first_var.get():
            # Switch turn to AI
            self.current_player = AI
            self.status_var.set("AI thinking…")
            self.root.update_idletasks()
            self.root.after(300, self.ai_move)

    def ai_move(self):
        if self.game_over:
            return
        _, move = minimax_alpha_beta(self.board, True, -math.inf, math.inf, 0, self.difficulty_depth())
        if move is not None:
            self.place_mark(move, AI)  # Always place AI's symbol
        winner, line = evaluate_winner(self.board)
        if winner:
            self.end_game(winner, line)
            return
        self.current_player = HUMAN
        if self.ai_first_var.get():
            self.root.after(300, self.ai_vs_ai_step)
        else:
            self.status_var.set("Your turn (X)")

    def ai_vs_ai_step(self):
        if self.game_over:
            return
        maximizing = (self.current_player == AI)
        _, move = minimax_alpha_beta(self.board, maximizing, -math.inf, math.inf, 0, self.difficulty_depth())
        if move is not None:
            self.place_mark(move, self.current_player)
        winner, line = evaluate_winner(self.board)
        if winner:
            self.end_game(winner, line)
            return
        self.current_player = HUMAN if self.current_player == AI else AI
        self.root.after(300, self.ai_vs_ai_step)

    def end_game(self, winner, line):
        self.game_over = True
        if line:
            for i in line:
                self.buttons[i].config(bg="#b2ffb2" if winner != "DRAW" else "#ddd")
        if winner == "DRAW":
            self.status_var.set("It's a draw!")
        else:
            self.status_var.set(f"{winner} wins!")

    def reset(self):
        self.board = [None] * 9
        self.move_history = []
        self.last_move_idx = None
        self.current_player = HUMAN
        self.game_over = False
        self.update_buttons()
        self.status_var.set("Your turn (X)" if not self.ai_first_var.get() else "AI vs AI running...")

    def undo(self):
        if not self.move_history or self.game_over:
            return
        idx, _ = self.move_history.pop()
        self.board[idx] = None
        if self.move_history:
            self.last_move_idx = self.move_history[-1][0]
        else:
            self.last_move_idx = None
        self.current_player = HUMAN
        self.update_buttons()
        self.status_var.set("Move undone.")


if __name__ == "__main__":
    root = tk.Tk()
    app = TicTacToeApp(root)
    root.mainloop()
